In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive = '/content/drive/MyDrive/rek_syst'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install surprise

In [ ]:
import pandas as pd
from surprise import Dataset, SVDpp, Reader, accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors
import numpy as np
algoritm_cv=SVDpp(n_factors=20,
                  n_epochs=20,
                  verbose=False,
                  cache_ratings=False)
def download():
    movies =  pd.read_csv(drive + '/ml-latest-small/movies.csv')
    rating = pd.read_csv(drive + '/ml-latest-small/ratings.csv')
    return movies,rating
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))
def pred_rating(uidx,iidx):
    ratingx = algoritm_cv.predict(uid=uidx, iid=iidx).est
    return ratingx
def start():
    movies,rating = download()
    mov_rat = movies.merge(rating, on='movieId').reset_index(drop=True).dropna()
    data = pd.DataFrame({
        'uid' : mov_rat.userId,
        'iid' : mov_rat.title,
        'rating' : mov_rat .rating})
    min_rating = data.rating.min()
    max_rating = data.rating.max()
    reader = Reader(rating_scale=(min_rating , max_rating))
    data_for_svdpp = Dataset.load_from_df(data,reader)
    trainset,testset = train_test_split(data_for_svdpp, test_size = 0.2,random_state = 13)
    algoritm_cv.fit(trainset)

    genres = {}
    for index, row in mov_rat.iterrows():
        genres[change_string(row.title)] = change_string(row.genres)

    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(genres)
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

    neigh = NearestNeighbors(n_neighbors=23, n_jobs=-1, metric='euclidean')
    neigh.fit(X_train_tfidf)
    return movies,rating,mov_rat,algoritm_cv,genres,count_vect,tfidf_transformer,neigh
def rec_for_usr(uidx):
    user_movies = mov_rat[mov_rat.userId == uidx].title.unique()
    meanratuidx = mov_rat[mov_rat.userId == uidx].rating.mean()
    user_movies_last = user_movies[-1]

    predict = count_vect.transform([user_movies_last])
    X_tfidf2 = tfidf_transformer.transform(predict)
    res = neigh.kneighbors(X_tfidf2, return_distance=True)

    movies_to_score = movies.iloc[res[1][0]].title.values
    scores = []
    titles = []
    for movie in movies_to_score:
        if movie in user_movies:
            continue
        a = algoritm_cv.predict(uid=uidx, iid=movie).est
        if a > meanratuidx:
            scores.append(algoritm_cv.predict(uid=uidx, iid=movie).est)
            titles.append(movie)
    for i in range(0, len(titles)):
            print(titles[i], round(scores[i],1))
    return

Собрал все подготовительное в один блок для удобства запуска, и сделал запуск через проверку условия(есть ли первая переменная в памяти) и ввод интересующего пользователя. "холодный" старт занимает 1-2 минуты на обучение, после несколько секунд.

In [ ]:
#"холодный" старт
%%time
uidx = int(input())
if 'mov_rat' in locals():
    print('ID пользователя: ',uidx)
    rec_for_usr(uidx)
else:
  movies,rating,mov_rat,algoritm_cv,genres,count_vect,tfidf_transformer,neigh = start()
  print('ID пользователя: ',uidx)
  rec_for_usr(uidx)

5
ID пользователя:  5
Whole Wide World, The (1996) 3.6
Alaska (1996) 3.7
Emma (1996) 4.0
Crucible, The (1996) 3.9
Trainspotting (1996) 4.1
CPU times: user 1min 30s, sys: 186 ms, total: 1min 31s
Wall time: 1min 37s


In [ ]:
#"обычный" старт
%%time
uidx = int(input())
if 'mov_rat' in locals():
    print('ID пользователя: ',uidx)
    rec_for_usr(uidx)
else:
  movies,rating,mov_rat,algoritm_cv,genres,count_vect,tfidf_transformer,neigh = start()
  print('ID пользователя: ',uidx)
  rec_for_usr(uidx)

5
ID пользователя:  5
Whole Wide World, The (1996) 3.6
Alaska (1996) 3.7
Emma (1996) 4.0
Crucible, The (1996) 3.9
Trainspotting (1996) 4.1
CPU times: user 27.6 ms, sys: 4 ms, total: 31.6 ms
Wall time: 2.6 s


In [ ]:
%%time
uidx = int(input())
if 'mov_rat' in locals():
    print('ID пользователя: ',uidx)
    rec_for_usr(uidx)
else:
  movies,rating,mov_rat,algoritm_cv,genres,count_vect,tfidf_transformer,neigh = start()
  print('ID пользователя: ',uidx)
  rec_for_usr(uidx)

99
ID пользователя:  99
Harakiri (Seppuku) (1962) 4.1
Footloose (2011) 4.1
Class Act (1992) 3.9
Whiplash (2014) 4.5
Watchmen (2009) 4.2
TiMER (2009) 4.1
Fast Times at Ridgemont High (1982) 4.0
From the Earth to the Moon (1998) 4.0
Playing for Keeps (2012) 4.0
Wind Rises, The (Kaze tachinu) (2013) 4.1
Platoon (1986) 4.5
He Said, She Said (1991) 3.9
Woodsman, The (2004) 4.2
Lean on Me (1989) 3.9
Big Sleep, The (1946) 4.4
Full Metal Jacket (1987) 4.5
CPU times: user 29.2 ms, sys: 1.03 ms, total: 30.3 ms
Wall time: 1.18 s
